In [1]:
#imports
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import pandas as pd
import numpy as np
import dask_ml
#prefs
pd.set_option('max_columns', None)


In [2]:
#close existing dask connection if it exists and open a new one
try:
    if client is not None:
        client.close()
        print("closed existing connection, ",client)       
except Exception as e:
    print(e)
    

name 'client' is not defined


In [3]:
#get new connection
client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
display(client)

Client Scheduler: tcp://127.0.0.1:41545 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 12.26 GiB


In [4]:
#get stock data
data = dd.read_csv("ids-data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv")

display(data.head())
#display(data.dtypes)
#data = progress(data)
#data
#client.persist(data)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,01/03/2018 08:17:11,115307855,5,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000,0.043362,28800000.0,32400000.0,61000000,812396,115000000,28800000.0,32400000.0,61000000,812396,0,0.0,0.0,0,0,0,0,0,0,0,0,0.043362,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000,Benign
1,0,0,01/03/2018 08:20:07,60997457,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000,0.032788,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000,Benign
2,67,17,01/03/2018 08:17:18,61149019,5,0,1500,0,300,300,300.0,0.0,0,0,0.0,0.0,24.530238,0.081767,15300000.0,12800000.0,32600000,3530939,61100000,15300000.0,12800000.0,32600000,3530939,0,0.0,0.0,0,0,0,0,0,0,40,0,0.081767,0.0,300,300,300.0,0.0,0.0,0,0,0,0,0,0,0,0,0,360.0,300.0,0.0,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725,Benign
3,0,0,01/03/2018 08:22:09,60997555,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000,0.032788,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000,Benign
4,0,0,01/03/2018 08:24:11,61997503,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000,0.048389,31000000.0,42400000.0,61000000,999909,62000000,31000000.0,42400000.0,61000000,999909,0,0.0,0.0,0,0,0,0,0,0,0,0,0.048389,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000,Benign


In [5]:
data['Timestamp'] = dd.to_datetime(data.Timestamp, format='%m/%d/%Y %H:%M:%S').astype(np.int64)
#data['Timestamp'] = dd.to_numeric(data.Timestamp)
tmp_data = data.loc[:, 'Dst Port':'Idle Min']
#import dask.array as da
#data = data[da.isfinite(data)]
#data['Timestamp'] = data.Timestamp + np.timedelta64(0)

In [9]:
too_big = (tmp_data > np.finfo(np.float32).max)
too_big = (too_big).any()
with pd.option_context('max_rows', None):
    display(too_big.compute())

Dst Port             False
Protocol             False
Timestamp            False
Flow Duration        False
Tot Fwd Pkts         False
Tot Bwd Pkts         False
TotLen Fwd Pkts      False
TotLen Bwd Pkts      False
Fwd Pkt Len Max      False
Fwd Pkt Len Min      False
Fwd Pkt Len Mean     False
Fwd Pkt Len Std      False
Bwd Pkt Len Max      False
Bwd Pkt Len Min      False
Bwd Pkt Len Mean     False
Bwd Pkt Len Std      False
Flow Byts/s           True
Flow Pkts/s           True
Flow IAT Mean        False
Flow IAT Std         False
Flow IAT Max         False
Flow IAT Min         False
Fwd IAT Tot          False
Fwd IAT Mean         False
Fwd IAT Std          False
Fwd IAT Max          False
Fwd IAT Min          False
Bwd IAT Tot          False
Bwd IAT Mean         False
Bwd IAT Std          False
Bwd IAT Max          False
Bwd IAT Min          False
Fwd PSH Flags        False
Bwd PSH Flags        False
Fwd URG Flags        False
Bwd URG Flags        False
Fwd Header Len       False
B

In [10]:
#get labels for decision tree presentation, note the use of unique later 
labels = data['Label']
labels

Dask Series Structure:
npartitions=2
    object
       ...
       ...
Name: Label, dtype: object
Dask Name: getitem, 12 tasks

In [11]:
type(data.head()['Timestamp'][0])

numpy.int64

In [12]:
len(labels)

#yD = p[['wentUp']]
#yD.head()

331098

In [13]:
# TODO: Figure out if we want to use a subset of available features. For now, use them all.
#features = ["Current Assets","Liabilities","Current Liabilities","Shareholders equity","Non-controlling interest","Preferred equity","Goodwill & intangibles"]

possible_features = 'Dst Port	Protocol	Timestamp	Flow Duration	Tot Fwd Pkts	Tot Bwd Pkts	TotLen Fwd Pkts	TotLen Bwd Pkts	Fwd Pkt Len Max	Fwd Pkt Len Min	Fwd Pkt Len Mean	Fwd Pkt Len Std	Bwd Pkt Len Max	Bwd Pkt Len Min	Bwd Pkt Len Mean	Bwd Pkt Len Std	Flow Byts/s	Flow Pkts/s	Flow IAT Mean	Flow IAT Std	Flow IAT Max	Flow IAT Min	Fwd IAT Tot	Fwd IAT Mean	Fwd IAT Std	Fwd IAT Max	Fwd IAT Min	Bwd IAT Tot	Bwd IAT Mean	Bwd IAT Std	Bwd IAT Max	Bwd IAT Min	Fwd PSH Flags	Bwd PSH Flags	Fwd URG Flags	Bwd URG Flags	Fwd Header Len	Bwd Header Len	Fwd Pkts/s	Bwd Pkts/s	Pkt Len Min	Pkt Len Max	Pkt Len Mean	Pkt Len Std	Pkt Len Var	FIN Flag Cnt	SYN Flag Cnt	RST Flag Cnt	PSH Flag Cnt	ACK Flag Cnt	URG Flag Cnt	CWE Flag Count	ECE Flag Cnt	Down/Up Ratio	Pkt Size Avg	Fwd Seg Size Avg	Bwd Seg Size Avg	Fwd Byts/b Avg	Fwd Pkts/b Avg	Fwd Blk Rate Avg	Bwd Byts/b Avg	Bwd Pkts/b Avg	Bwd Blk Rate Avg	Subflow Fwd Pkts	Subflow Fwd Byts	Subflow Bwd Pkts	Subflow Bwd Byts	Init Fwd Win Byts	Init Bwd Win Byts	Fwd Act Data Pkts	Fwd Seg Size Min	Active Mean	Active Std	Active Max	Active Min	Idle Mean	Idle Std	Idle Max	Idle Min'

features = [f for f in possible_features.split('\t') if f not in ('Timestamp', 'Flow Byts/s', 'Flow Pkts/s')]


In [14]:
#get dataframe of just features
X = data.loc[:, features]
X.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,28800000.0,32400000.0,61000000,812396,115000000,28800000.0,32400000.0,61000000,812396,0,0.0,0.0,0,0,0,0,0,0,0,0,0.043362,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
1,0,0,60997457,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
2,67,17,61149019,5,0,1500,0,300,300,300.0,0.0,0,0,0.0,0.0,15300000.0,12800000.0,32600000,3530939,61100000,15300000.0,12800000.0,32600000,3530939,0,0.0,0.0,0,0,0,0,0,0,40,0,0.081767,0.0,300,300,300.0,0.0,0.0,0,0,0,0,0,0,0,0,0,360.0,300.0,0.0,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,31000000.0,42400000.0,61000000,999909,62000000,31000000.0,42400000.0,61000000,999909,0,0.0,0.0,0,0,0,0,0,0,0,0,0.048389,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000


In [15]:
#setup plot
import matplotlib.pyplot as plt
print(plt.rcParams.get('figure.figsize'))

[6.0, 4.0]


In [16]:
#setup figure size
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 20
plt.rcParams["figure.figsize"] = fig_size

In [17]:
Y = labels
Y.head()

0    Benign
1    Benign
2    Benign
3    Benign
4    Benign
Name: Label, dtype: object

In [18]:
from platform import python_version
print(python_version())
import sklearn

3.8.5


In [19]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))
from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=0)
#one_hot_data = pd.get_dummies(p[features],drop_first=True)
#print(one_hot_data)cc
#clf = clf.fit(one_hot_data, Y)
import joblib
#get pandas dataframes to test
x = X.compute()
y = Y.compute()

The scikit-learn version is 0.24.1.


In [20]:
display(X.head())
print(type(X))
display(Y.head())
print(type(Y))

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,28800000.0,32400000.0,61000000,812396,115000000,28800000.0,32400000.0,61000000,812396,0,0.0,0.0,0,0,0,0,0,0,0,0,0.043362,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
1,0,0,60997457,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
2,67,17,61149019,5,0,1500,0,300,300,300.0,0.0,0,0,0.0,0.0,15300000.0,12800000.0,32600000,3530939,61100000,15300000.0,12800000.0,32600000,3530939,0,0.0,0.0,0,0,0,0,0,0,40,0,0.081767,0.0,300,300,300.0,0.0,0.0,0,0,0,0,0,0,0,0,0,360.0,300.0,0.0,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,31000000.0,42400000.0,61000000,999909,62000000,31000000.0,42400000.0,61000000,999909,0,0.0,0.0,0,0,0,0,0,0,0,0,0.048389,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000


<class 'dask.dataframe.core.DataFrame'>


0    Benign
1    Benign
2    Benign
3    Benign
4    Benign
Name: Label, dtype: object

<class 'dask.dataframe.core.Series'>


In [21]:
#setup Dask kfolds
#pip install dask_ml
#conda install dask_ml
from dask_ml.model_selection import KFold as K
from dask_ml.model_selection import train_test_split as ts


In [22]:
%%time
#function that takes X, Y and gives you a train test split
def doTrainTestSplit(X,Y):
    #breakpoint()
    #look at dask dataframes
    display(X.head())
    display(Y.head())
   
    #get splits
    X_train, X_test, y_train, y_test = ts(X,Y,test_size=.5)
    display(X_train.compute())
    display(y_train.compute())
    
    #do some inspection
    print("X_test")
    display(X_test.compute())
    display(y_test.compute())
    
    #fit and predict
    print(clf.fit(X_train.compute(),y_train.compute()))
    print(clf.predict(X_test.compute()))
    
    return clf
        

doTrainTestSplit(X,Y)



,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,28800000.0,32400000.0,61000000,812396,115000000,28800000.0,32400000.0,61000000,812396,0,0.0,0.0,0,0,0,0,0,0,0,0,0.043362,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
1,0,0,60997457,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
2,67,17,61149019,5,0,1500,0,300,300,300.0,0.0,0,0,0.0,0.0,15300000.0,12800000.0,32600000,3530939,61100000,15300000.0,12800000.0,32600000,3530939,0,0.0,0.0,0,0,0,0,0,0,40,0,0.081767,0.0,300,300,300.0,0.0,0.0,0,0,0,0,0,0,0,0,0,360.0,300.0,0.0,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,31000000.0,42400000.0,61000000,999909,62000000,31000000.0,42400000.0,61000000,999909,0,0.0,0.0,0,0,0,0,0,0,0,0,0.048389,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000


0    Benign
1    Benign
2    Benign
3    Benign
4    Benign
Name: Label, dtype: object

/home/mwdehn/.local/lib/python3.8/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,2.880000e+07,3.240000e+07,61000000,812396,115000000,2.880000e+07,3.240000e+07,61000000,812396,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.043362,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
5,0,0,61997503,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,3.100000e+07,4.240000e+07,61000000,1000011,62000000,3.100000e+07,4.240000e+07,61000000,1000011,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.048389,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
7,0,0,60997542,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
8,0,0,60997552,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
9,0,0,60997614,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133910,46499,6,86373330,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,8.640000e+07,0.000000e+00,86400000,86400000,86400000,8.640000e+07,0.000000e+00,86400000,86400000,0,0.0000,0.0000,0,0,0,0,0,0,40,0,0.023155,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,62608,-1,0,20,0.0,0.0,0,0,86400000.0,0.000000e+00,86400000,86400000
133912,3389,6,4764063,9,7,1148,1581,677,0,127.555556,218.691056,1173,0,225.857143,430.098604,3.176042e+05,4.901504e+05,1803816,55,4764063,5.955079e+05,6.146567e+05,1803816,167,4525752,754292.0000,807965.8382,2309472,240363,0,0,0,0,192,152,1.889144,1.469334,0,1173,160.529412,312.044091,97371.51471,0,0,0,1,0,0,0,0,0,170.562500,127.555556,225.857143,0,0,0,0,0,0,9,1148,7,1581,8192

0         Benign
5         Benign
7         Benign
8         Benign
9         Benign
           ...  
133910    Benign
133912    Benign
133914    Benign
133918    Benign
133922    Benign
Name: Label, Length: 164996, dtype: object

X_test


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
1,0,0,60997457,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.00,0,0,61000000.0,0.000000e+00,61000000,61000000
2,67,17,61149019,5,0,1500,0,300,300,300.000000,0.000000,0,0,0.000000,0.000000,1.530000e+07,1.280000e+07,32600000,3530939,61100000,1.530000e+07,1.280000e+07,32600000,3530939,0,0.0000,0.0000,0,0,0,0,0,0,40,0,0.081767,0.000000,300,300,300.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,360.000000,300.000000,0.000000,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.00,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.00,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,3.100000e+07,4.240000e+07,61000000,999909,62000000,3.100000e+07,4.240000e+07,61000000,999909,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.048389,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.00,999909,999909,61000000.0,0.000000e+00,61000000,61000000
6,67,17,92344822,9,0,2700,0,300,300,300.000000,0.000000,0,0,0.000000,0.000000,1.150000e+07,9.387454e+06,31300000,3999448,92300000,1.150000e+07,9.387454e+06,31300000,3999448,0,0.0000,0.0000,0,0,0,0,0,0,72,0,0.097461,0.000000,300,300,300.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,333.333333,300.000000,0.000000,0,0,0,0,0,0,9,2700,0,0,-1,-1,8,8,6678774.0,2928068.37,8749231,4608317,15800000.0,9.685366e+06,31300000,7468536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133916,0,0,112645069,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,5.630000e+07,3.582274e+04,56300000,56300000,113000000,5.630000e+07,3.582274e+04,56300000,56300000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.026632,0.000000,0,0,0.000000,0.000000,0.00000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.00,0,0,56300000.0,3.582274e+04,56300000,56300000
133917,3389,6,3866185,13,7,1392,392,684,0,107.076923,195.613335,291,0,56.000000,105.397660,2.034834e+05,2.574307e+05,953214,28,3866185,3.221821e+05,3.422473e+05,1228038,47681,3813241,635540.1667,309869.8643,953214,166333,0,0,0,0,272,152,3.362488,1.810570,0,684,84.952381,165.117375,27263.74762,0,0

1         Benign
2         Benign
3         Benign
4         Benign
6         Benign
           ...  
133916    Benign
133917    Benign
133919    Benign
133920    Benign
133921    Benign
Name: Label, Length: 166102, dtype: object

DecisionTreeClassifier(random_state=0)
['Benign' 'Benign' 'Benign' ... 'Infilteration' 'Benign' 'Benign']
CPU times: user 27 s, sys: 1.99 s, total: 29 s
Wall time: 1min 18s


DecisionTreeClassifier(random_state=0)

In [23]:
%%time
def doTrainTestSplitDask(X,Y):
    #breakpoint()
    #look at dask dataframes
    display(X.head())
    display(Y.head())
   
    #get splits
    X_train, X_test, y_train, y_test = ts(X,Y,test_size=.5)
    display(X_train.compute())
    display(y_train.compute())
    
    #do some inspection
    print("X_test")
    display(X_test.compute())
    display(y_test.compute())
    
    #fit and predict
    print(clf.fit(X_train,y_train))
    print(clf.predict(X_test))
    
    return clf
        

doTrainTestSplitDask(X,Y)

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,28800000.0,32400000.0,61000000,812396,115000000,28800000.0,32400000.0,61000000,812396,0,0.0,0.0,0,0,0,0,0,0,0,0,0.043362,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
1,0,0,60997457,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
2,67,17,61149019,5,0,1500,0,300,300,300.0,0.0,0,0,0.0,0.0,15300000.0,12800000.0,32600000,3530939,61100000,15300000.0,12800000.0,32600000,3530939,0,0.0,0.0,0,0,0,0,0,0,40,0,0.081767,0.0,300,300,300.0,0.0,0.0,0,0,0,0,0,0,0,0,0,360.0,300.0,0.0,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,61000000.0,0.0,61000000,61000000,61000000,61000000.0,0.0,61000000,61000000,0,0.0,0.0,0,0,0,0,0,0,0,0,0.032788,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,31000000.0,42400000.0,61000000,999909,62000000,31000000.0,42400000.0,61000000,999909,0,0.0,0.0,0,0,0,0,0,0,0,0,0.048389,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000


0    Benign
1    Benign
2    Benign
3    Benign
4    Benign
Name: Label, dtype: object

/home/mwdehn/.local/lib/python3.8/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
1,0,0,60997457,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.00000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.000,0,0,61000000.0,0.000000e+00,61000000,61000000
6,67,17,92344822,9,0,2700,0,300,300,300.000000,0.000000,0,0,0.000000,0.000000,1.150000e+07,9.387454e+06,31300000,3999448,92300000,1.150000e+07,9.387454e+06,31300000,3999448,0,0.0000,0.00000,0,0,0,0,0,0,72,0,0.097461,0.000000,300,300,300.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,333.333333,300.000000,0.000000,0,0,0,0,0,0,9,2700,0,0,-1,-1,8,8,6678774.0,2928068.370,8749231,4608317,15800000.0,9.685366e+06,31300000,7468536
10,0,0,60997547,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.00000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.000,0,0,61000000.0,0.000000e+00,61000000,61000000
11,67,17,94865721,9,0,2700,0,300,300,300.000000,0.000000,0,0,0.000000,0.000000,1.190000e+07,9.759822e+06,32500000,3984222,94900000,1.190000e+07,9.759822e+06,32500000,3984222,0,0.0000,0.00000,0,0,0,0,0,0,72,0,0.094871,0.000000,300,300,300.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,333.333333,300.000000,0.000000,0,0,0,0,0,0,9,2700,0,0,-1,-1,8,8,6504919.0,3477971.171,8964216,4045622,16400000.0,9.932252e+06,32500000,7984192
12,0,0,60997585,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.00000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.000,0,0,61000000.0,0.000000e+00,61000000,61000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133913,445,6,538639,5,6,373,172,140,0,74.600000,70.283711,133,0,28.666667,53.440309,5.386390e+04,4.829191e+04,112068,4,538552,1.346380e+05,1.086222e+05,240746,4,538586,107717.2000,76055.71914,203105,139,0,0,0,0,128,132,9.282655,11.139186,0,140,45.416667,61.823000,3822.083333,0,0,0,1,0,0,0,0,1,49.545455,74.600000,28.666667,0,0,0,0,0,0,5,373,6,172,8192,0,3,20,0.0,0.000,0,0,0.0,0.000000e+00,0,0
133914,0,0,112640353,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,5.630000e+07,6.858936e+01,56300000,56300000,113000000,5.630000e+07,6.858936e+01,56300000,56300000,0,0.0000,0.00000,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0

1         Benign
6         Benign
10        Benign
11        Benign
12        Benign
           ...  
133913    Benign
133914    Benign
133917    Benign
133918    Benign
133922    Benign
Name: Label, Length: 165623, dtype: object

X_test


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,115307855,5,0,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,2.880000e+07,3.240000e+07,61000000,812396,115000000,2.880000e+07,3.240000e+07,61000000,812396,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.043362,0.000000,0,0,0.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,0,0,0,0,0,5,0,0,0,-1,-1,0,0,1812348.0,0.0,1812348,1812348,56700000.0,6.010058e+06,61000000,52500000
2,67,17,61149019,5,0,1500,0,300,300,300.0,0.000000,0,0,0.000000,0.000000,1.530000e+07,1.280000e+07,32600000,3530939,61100000,1.530000e+07,1.280000e+07,32600000,3530939,0,0.0000,0.0000,0,0,0,0,0,0,40,0,0.081767,0.000000,300,300,300.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,360.0,300.0,0.000000,0,0,0,0,0,0,5,1500,0,0,-1,-1,4,8,3530939.0,0.0,3530939,3530939,19200000.0,1.250000e+07,32600000,7999725
3,0,0,60997555,2,0,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,6.100000e+07,0.000000e+00,61000000,61000000,61000000,6.100000e+07,0.000000e+00,61000000,61000000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.032788,0.000000,0,0,0.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,0,0,0,0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
4,0,0,61997503,3,0,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,3.100000e+07,4.240000e+07,61000000,999909,62000000,3.100000e+07,4.240000e+07,61000000,999909,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.048389,0.000000,0,0,0.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,999909.0,0.0,999909,999909,61000000.0,0.000000e+00,61000000,61000000
5,0,0,61997503,3,0,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,3.100000e+07,4.240000e+07,61000000,1000011,62000000,3.100000e+07,4.240000e+07,61000000,1000011,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.048389,0.000000,0,0,0.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,61000000.0,0.000000e+00,61000000,61000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133915,3389,6,1676774,8,7,1128,1581,661,0,141.0,222.623321,1173,0,225.857143,430.098604,1.197696e+05,2.413389e+05,937520,3,1676774,2.395391e+05,3.861941e+05,1109979,3,1567658,261276.3333,332154.3051,937520,109382,0,0,0,0,172,152,4.771066,4.174683,0,1173,169.3125,317.731274,100953.1625,0,0,1,1,0,0,0,1,0,180.6,141.0,225.857143,0,0,0,0,0,0,8,1128,7,1581,8192,62872,5,20,0.0,0.0,0,0,0.0,0.000000e+00,0,0
133916,0,0,112645069,3,0,0,0,0,0,0.0,0.000000,0,0,0.000000,0.000000,5.630000e+07,3.582274e+04,56300000,56300000,113000000,5.630000e+07,3.582274e+04,56300000,56300000,0,0.0000,0.0000,0,0,0,0,0,0,0,0,0.026632,0.000000,0,0,0.0000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56300000.0,3.582274e+04,56300000,56300000
133919,3389,6,1721142,8,7,1128,1581,661,0

0         Benign
2         Benign
3         Benign
4         Benign
5         Benign
           ...  
133915    Benign
133916    Benign
133919    Benign
133920    Benign
133921    Benign
Name: Label, Length: 165475, dtype: object

DecisionTreeClassifier(random_state=0)
['Infilteration' 'Benign' 'Benign' ... 'Benign' 'Benign' 'Benign']
CPU times: user 26.9 s, sys: 1.7 s, total: 28.6 s
Wall time: 1min 20s


DecisionTreeClassifier(random_state=0)

In [24]:

def doKFolds(X,Y):
    
    #look at dask dataframes
    display(X.head())
    display(yD.head())

    #create KFold object
    c = K()
    #breakpoint()
    #split on dask arrays, doesn't work on dataframes yet
    gen = c.split(X.to_dask_array(lengths=True),Y.to_dask_array(lengths=True))

    #inspect generator
    print(gen)
    display(type(gen))
    
    #call generator
    for train,test in gen:
        print("train = ",train.compute())
        print("test = ",test.compute())
        print("x train = ",X.loc[train])
        got = X.loc[train.compute()]
        display(got.head())
        #print(got.compute())
        clf.fit(got,got)

        
doKFolds(X,yD)

NameError: name 'yD' is not defined

In [ ]:
# import pandas as pd
# import dask.dataframe as dd

#generate example dataframe
pdf = pd.DataFrame(dict(A = [1,2,3,4,5], B = [6,7,8,9,0]), index=['i1', 'i2', 'i3', '4', '5'])
#ddf = dd.from_pandas(pdf, npartitions = 2,)
ddf = dd.from_pandas(pdf,chunksize=int(len(pdf)/2))

# #list of indices I want to select
l = ['i1', '4', '5']

# #generate new dask dataframe containing only the specified indices
# ddf_selected = ddf.map_partitions(lambda x: x[x.index.isin(l)], meta = ddf.dtypes)
ddf_selected = ddf.loc[l]
ddf_selected.head()

In [ ]:
%%timeit -n 10
#time with just pandas
with joblib.parallel_backend('threading'):
     clf.fit(x, y)


In [ ]:
%%timeit -n 10
#time with dask
with joblib.parallel_backend('dask'):
     clf.fit(X, Y)


#clf = clf.fit(X, Y)

In [ ]:
clf.fit(X, Y)


In [ ]:
%%timeit -n 10
#time with dask, even if you set threaing it's still a dask dataframe so it's produced via dasks engine
with joblib.parallel_backend('threading'):
     clf.fit(X, Y)


#clf = clf.fit(X, Y)

In [ ]:
#cache results note you have to assign in X and Y
X = client.persist(X)
Y = client.persist(Y)

In [ ]:
%%timeit -n 10
#time with dask after cache
with joblib.parallel_backend('dask'):
     clf.fit(X, Y)

In [ ]:
%%timeit -n 10
#time with just pandas once X and Y are cached
with joblib.parallel_backend('threading'):
     clf.fit(x, y)

In [ ]:
#get sorted labels for plot 
import numpy as np
sorted = labels.unique()
sorted = np.sort(sorted)
sorted = list(map(str, sorted))
sorted

In [ ]:
pd.set_option('display.max_columns', None)  
#pd.get_dummies(p)

In [ ]:
features
#p['Current Assets']

In [ ]:
#x = tree.plot_tree(clf,feature_names=features,class_names=labels.astype(str),rounded=True,filled=True) 
x = tree.plot_tree(clf,rounded=True,filled=True,class_names=sorted,feature_names=features) 


In [ ]:
#client.close()